Get weather data for the Dataset

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

In [2]:
# Add the directory containing the module to the system path
module_path = os.path.abspath(os.path.join('C:\\Users\\samue\\OneDrive\\AIML\\HS2024\\Data Sicence Projekt\\HSLU_DSPRO1_TrafficStatus\\data'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the module
import DataEngineeringLibrary as dlib

In [6]:
dataLondonUTD19 = dlib.loadData(path=r"C:\Users\samue\OneDrive\AIML\HS2024\Data Sicence Projekt\Data\London_UTD19.csv", nrows=1000000)
dataLondonUTD19.head()

,day,interval,detid,flow,occ,error,city,speed
0,2015-05-16,0,EAST_N04/161x1,144.0,0.028333,0.0,london,NaN
1,2015-05-16,300,EAST_N04/161x1,204.0,0.040000,0.0,london,NaN
2,2015-05-16,600,EAST_N04/161x1,216.0,0.039167,0.0,london,NaN
3,2015-05-16,900,EAST_N04/161x1,168.0,0.029167,0.0,london,NaN
4,2015-05-16,1200,EAST_N04/161x1,144.0,0.024167,0.0,london,NaN


In [24]:
unique_days = dataLondonUTD19['day'].unique()
unique_days = pd.to_datetime(unique_days)
unique_days

DatetimeIndex(['2015-05-16', '2015-05-15', '2015-05-17', '2015-05-18',
               '2015-05-19', '2015-05-20', '2015-05-21', '2015-05-22',
               '2015-05-23', '2015-10-01', '2015-09-25', '2015-09-26',
               '2015-09-27', '2015-09-28', '2015-09-29', '2015-09-30',
               '2016-05-16', '2016-05-17', '2016-05-18', '2016-05-19',
               '2016-05-20', '2016-05-21', '2016-05-22'],
              dtype='datetime64[ns]', freq=None)

In [14]:
#This is the example from the Open-Meteo API documentation
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 51.50,
	"longitude": -0.12,
	"start_date": "2015-10-14",
	"end_date": "2015-10-15",
	"hourly": "rain"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_rain = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["rain"] = hourly_rain

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 51.49384689331055°N -0.16302490234375°E
Elevation 1.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                        date  rain
0  2015-10-14 00:00:00+00:00   0.0
1  2015-10-14 01:00:00+00:00   0.0
2  2015-10-14 02:00:00+00:00   0.0
3  2015-10-14 03:00:00+00:00   0.0
4  2015-10-14 04:00:00+00:00   0.0
5  2015-10-14 05:00:00+00:00   0.0
6  2015-10-14 06:00:00+00:00   0.0
7  2015-10-14 07:00:00+00:00   0.0
8  2015-10-14 08:00:00+00:00   0.0
9  2015-10-14 09:00:00+00:00   0.0
10 2015-10-14 10:00:00+00:00   0.0
11 2015-10-14 11:00:00+00:00   0.0
12 2015-10-14 12:00:00+00:00   0.0
13 2015-10-14 13:00:00+00:00   0.0
14 2015-10-14 14:00:00+00:00   0.0
15 2015-10-14 15:00:00+00:00   0.1
16 2015-10-14 16:00:00+00:00   0.0
17 2015-10-14 17:00:00+00:00   0.0
18 2015-10-14 18:00:00+00:00   0.0
19 2015-10-14 19:00:00+00:00   0.0
20 2015-10-14 20:00:00+00:00   0.0
21 2015-10-14 21:00:00+00:00   0.0
22 2015-10-14 22:00:00+00:00   0.0
23 2015-10-14 23:00:00+00:00   0.0
2

In [30]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
url = "https://archive-api.open-meteo.com/v1/archive"

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each unique day value
for day in unique_days:
    # Convert day to string format for API call
    day_str = day.strftime('%Y-%m-%d')
    
    # Set the parameters for the API call
    params = {
        "latitude": 51.50,
        "longitude": -0.12,
        "start_date": day_str,
        "end_date": day_str,
        "hourly": "rain"
    }
    
    # Call the API
    responses = openmeteo.weather_api(url, params=params)
    
    # Process the response
    response = responses[0]
    
    # Process hourly data
    hourly = response.Hourly()
    hourly_rain = hourly.Variables(0).ValuesAsNumpy()
    
    hourly_data = {"date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )}
    hourly_data["rain"] = hourly_rain
    
    hourly_dataframe = pd.DataFrame(data=hourly_data)
    print(hourly_dataframe)
    
    # Append the DataFrame to the list
    dataframes.append(hourly_dataframe)

# Combine all DataFrames into a single DataFrame
rainDataFrameResponse = pd.concat(dataframes, ignore_index=True)

rainDataFrameResponse.head()

                        date  rain
0  2015-05-16 00:00:00+00:00   0.0
1  2015-05-16 01:00:00+00:00   0.0
2  2015-05-16 02:00:00+00:00   0.0
3  2015-05-16 03:00:00+00:00   0.0
4  2015-05-16 04:00:00+00:00   0.0
5  2015-05-16 05:00:00+00:00   0.0
6  2015-05-16 06:00:00+00:00   0.0
7  2015-05-16 07:00:00+00:00   0.0
8  2015-05-16 08:00:00+00:00   0.0
9  2015-05-16 09:00:00+00:00   0.0
10 2015-05-16 10:00:00+00:00   0.0
11 2015-05-16 11:00:00+00:00   0.0
12 2015-05-16 12:00:00+00:00   0.0
13 2015-05-16 13:00:00+00:00   0.0
14 2015-05-16 14:00:00+00:00   0.0
15 2015-05-16 15:00:00+00:00   0.0
16 2015-05-16 16:00:00+00:00   0.0
17 2015-05-16 17:00:00+00:00   0.0
18 2015-05-16 18:00:00+00:00   0.0
19 2015-05-16 19:00:00+00:00   0.0
20 2015-05-16 20:00:00+00:00   0.0
21 2015-05-16 21:00:00+00:00   0.0
22 2015-05-16 22:00:00+00:00   0.0
23 2015-05-16 23:00:00+00:00   0.0
                        date  rain
0  2015-05-15 00:00:00+00:00   0.0
1  2015-05-15 01:00:00+00:00   0.0
2  2015-05-15 02:00:

,date,rain
0,2015-05-16 00:00:00+00:00,0.0
1,2015-05-16 01:00:00+00:00,0.0
2,2015-05-16 02:00:00+00:00,0.0
3,2015-05-16 03:00:00+00:00,0.0
4,2015-05-16 04:00:00+00:00,0.0


In [31]:
rainDataFrame = rainDataFrameResponse

In [32]:
# Convert the 'date' column to datetime (replace 'date' with your actual column name)
rainDataFrame['date'] = pd.to_datetime(rainDataFrame['date'])

# Extract date as string and time interval
rainDataFrame['date_string'] = rainDataFrame['date'].dt.strftime('%Y-%m-%d')
#rainDataFrame['interval'] = rainDataFrame['date'].dt.strftime('%H:%M:%S')
rainDataFrame['interval'] = rainDataFrame['date'].dt.hour * 3600 + rainDataFrame['date'].dt.minute * 60 + rainDataFrame['date'].dt.second

rainDataFrame.drop(columns=['date'], inplace=True)

rainDataFrame.head()

,rain,date_string,interval
0,0.0,2015-05-16,0
1,0.0,2015-05-16,3600
2,0.0,2015-05-16,7200
3,0.0,2015-05-16,10800
4,0.0,2015-05-16,14400


In [33]:
rainDataFrame.describe()

,rain,interval
count,552.000000,552.000000
mean,0.044022,41400.000000
std,0.205991,24942.474652
min,0.000000,0.000000
25%,0.000000,20700.000000
50%,0.000000,41400.000000
75%,0.000000,62100.000000
max,2.500000,82800.000000
